# Copyright Netherlands eScience Center <br>
** Function     : Take the maximum likelihood from trained BayesConvLSTM and save the weights to ConvLSTM** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.10.19 ** <br>
** Last Update  : 2020.10.19 ** <br>
** Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib **<br>
Description     : This notebook serves to test the fixed sampling of BayesConvLSTM and the initialization of ConvLSTM weight. This could help to test the sampling weight of a trained BayesConvLSTM neural network. <br>

Return Values   : pkl model <br>

In [1]:
%matplotlib inline

import sys
import warnings
import numbers

# for data loading
import os
from netCDF4 import Dataset
# for pre-processing and machine learning
import numpy as np
#import sklearn
#import scipy
import torch
import torch.nn.functional

#sys.path.append(os.path.join('C:','Users','nosta','ML4Climate','Scripts','DLACs'))
sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
import dlacs
import dlacs.BayesConvLSTM
import dlacs.ConvLSTM
import dlacs.preprocess
import dlacs.function
import dlacs.saveNetCDF

# for visualization
import dlacs.visual
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import iris # also helps with regriding
import cartopy
import cartopy.crs as ccrs

# ignore all the DeprecationWarnings by pytorch
if not sys.warnoptions:
    warnings.simplefilter("ignore")

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
################################################################################# 
#########                           modelpath                            ########
#################################################################################
# path of initialization weight
bayesconvlstm_model_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84\\bayes\\fullseries\\penalty100'
# path of output
output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps'

In [3]:
if __name__=="__main__":
    print ('*******************  create basic dimensions for network  *********************')
    # specifications of neural network
    input_channels = 3
    #hidden_channels = [3, 2, 1] # number of channels & hidden layers, the channels of last layer is the channels of output, too
    #hidden_channels = [3, 3, 3, 3, 2]
    hidden_channels = [3]
    kernel_size = 1
    batch_size = 1
    #num_layers = 1
    learning_rate = 0.01
    num_epochs = 1500

*******************  create basic dimensions for network  *********************


In [4]:
    print ('*******************  check the environment  *********************')
    print ("Pytorch version {}".format(torch.__version__))
    # check if CUDA is available
    use_cuda = torch.cuda.is_available()
    print("Is CUDA available? {}".format(use_cuda))
    # CUDA settings torch.__version__ must > 0.4
    # !!! This is important for the model!!! The first option is gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

*******************  check the environment  *********************
Pytorch version 1.1.0
Is CUDA available? True


In [5]:
    %%time
    print ('*******************  load exsited LSTM model  *********************')
    # load model parameters
    model_bayes = dlacs.BayesConvLSTM.BayesConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    model_bayes.load_state_dict(torch.load(os.path.join(bayesconvlstm_model_path, 'bayesconvlstm_lorenz84_fullseries_penalty100.pkl'),
                                             map_location=device))
    print(model_bayes)

*******************  load exsited LSTM model  *********************
!@#$% The network will be built with reduced size BayesConvLSTM cell. !@#$%
BayesConvLSTM(
  (cell0): BayesConvLSTMCell()
)
Wall time: 2.01 s


In [6]:
    print ('*******************  check wieght matrix of BayesConvLSTM  *********************')
    for name, param in model_bayes.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check wieght matrix of BayesConvLSTM  *********************
cell0.Wxi_mu
tensor([[[[ 0.2468]],

         [[-0.2178]],

         [[-0.1384]]],


        [[[ 0.1433]],

         [[-0.2203]],

         [[-0.1872]]],


        [[[ 0.1793]],

         [[-0.1780]],

         [[ 0.2258]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Whi_mu
tensor([[[[ 0.1465]],

         [[ 0.1269]],

         [[-0.1540]]],


        [[[-0.0113]],

         [[ 0.1824]],

         [[-0.1429]]],


        [[[ 0.1443]],

         [[-0.1535]],

         [[ 0.1442]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf_mu
tensor([[[[ 0.1686]],

         [[-0.1662]],

         [[-0.1287]]],


        [[[-0.1784]],

         [[-0.1708]],

         [[ 0.1272]]],


        [[[-0.1464]],

         [[ 0.1670]],

         [[ 0.1486]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Whf_mu
tensor([[[[ 0.7204]],

         [[ 1.5962]],

         [[ 0.8453]]],


        [[[ 0.1172]],

         [[

In [7]:
    print ('*******************  extract the weight from BayesConvLSTM  *********************')
    weight_model_BayesConvLSTM = {}
    # the weights are pytorch tensors
    for name, param in model_bayes.named_parameters():
        weight_model_BayesConvLSTM['{}'.format(name)] = param.data
        
    print ('*******************  check the weight from ConvLSTM  *********************')
    for i in weight_model_BayesConvLSTM:
        print('{}'.format(i))
        print('{}'.format(weight_model_BayesConvLSTM['{}'.format(i)]))
        print ("=========================")    

*******************  extract the weight from BayesConvLSTM  *********************
*******************  check the weight from ConvLSTM  *********************
cell0.Wxi_mu
tensor([[[[ 0.2468]],

         [[-0.2178]],

         [[-0.1384]]],


        [[[ 0.1433]],

         [[-0.2203]],

         [[-0.1872]]],


        [[[ 0.1793]],

         [[-0.1780]],

         [[ 0.2258]]]], device='cuda:0')
cell0.Whi_mu
tensor([[[[ 0.1465]],

         [[ 0.1269]],

         [[-0.1540]]],


        [[[-0.0113]],

         [[ 0.1824]],

         [[-0.1429]]],


        [[[ 0.1443]],

         [[-0.1535]],

         [[ 0.1442]]]], device='cuda:0')
cell0.Wxf_mu
tensor([[[[ 0.1686]],

         [[-0.1662]],

         [[-0.1287]]],


        [[[-0.1784]],

         [[-0.1708]],

         [[ 0.1272]]],


        [[[-0.1464]],

         [[ 0.1670]],

         [[ 0.1486]]]], device='cuda:0')
cell0.Whf_mu
tensor([[[[ 0.7204]],

         [[ 1.5962]],

         [[ 0.8453]]],


        [[[ 0.1172]],

         [

In [8]:
    ################################################################################# 
    #########            sample the weight of BayesConvLSTM                  ########
    #################################################################################
    namelist_weight = ['Wxi', 'Whi', 'Wxf', 'Whf', 'Wxc', 'Whc', 'Wxo', 'Who']
    namelist_bias = ['Wxi', 'Wxf', 'Wxc', 'Wxo']
    cell_index = len(hidden_channels)
    
    weight_model_init = {}
    for s in range(cell_index):
        for w in namelist_weight:
            # take the mean
            mu = weight_model_BayesConvLSTM['cell{}.{}_mu'.format(s, w)].data
            weight = mu
            weight_model_init['cell{}.{}.weight'.format(s, w)] = weight.data
        for b in namelist_bias:
            weight_model_init['cell{}.{}.bias'.format(s, b)] = weight_model_BayesConvLSTM['cell{}.{}_bias'.format(s, b)].data
    # create ConvLSTM network
    model_convlstm = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size,
                                             weight_dict = weight_model_init).to(device)
    # save the ConvLSTM sample
    torch.save(model_convlstm.state_dict(), os.path.join(output_path,'maximum_likelihood_convlstm_lorenz84.pkl'))

In [9]:
    print ('*******************  check the weight calculated weight  *********************')
    for i in weight_model_init:
        print('{}'.format(i))
        print('{}'.format(weight_model_init['{}'.format(i)]))
        print ("=========================")

*******************  check the weight calculated weight  *********************
cell0.Wxi.weight
tensor([[[[ 0.2468]],

         [[-0.2178]],

         [[-0.1384]]],


        [[[ 0.1433]],

         [[-0.2203]],

         [[-0.1872]]],


        [[[ 0.1793]],

         [[-0.1780]],

         [[ 0.2258]]]], device='cuda:0')
cell0.Whi.weight
tensor([[[[ 0.1465]],

         [[ 0.1269]],

         [[-0.1540]]],


        [[[-0.0113]],

         [[ 0.1824]],

         [[-0.1429]]],


        [[[ 0.1443]],

         [[-0.1535]],

         [[ 0.1442]]]], device='cuda:0')
cell0.Wxf.weight
tensor([[[[ 0.1686]],

         [[-0.1662]],

         [[-0.1287]]],


        [[[-0.1784]],

         [[-0.1708]],

         [[ 0.1272]]],


        [[[-0.1464]],

         [[ 0.1670]],

         [[ 0.1486]]]], device='cuda:0')
cell0.Whf.weight
tensor([[[[ 0.7204]],

         [[ 1.5962]],

         [[ 0.8453]]],


        [[[ 0.1172]],

         [[ 2.7743]],

         [[ 0.6885]]],


        [[[-0.0834]],

 

In [10]:
    print ('*******************  check the weight from ConvLSTM  *********************')
    for name, param in model_convlstm.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 0.2468]],

         [[-0.2178]],

         [[-0.1384]]],


        [[[ 0.1433]],

         [[-0.2203]],

         [[-0.1872]]],


        [[[ 0.1793]],

         [[-0.1780]],

         [[ 0.2258]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxi.bias
tensor([ 0.1563,  0.2719, -0.0114], device='cuda:0')
torch.Size([3])
cell0.Whi.weight
tensor([[[[ 0.1465]],

         [[ 0.1269]],

         [[-0.1540]]],


        [[[-0.0113]],

         [[ 0.1824]],

         [[-0.1429]]],


        [[[ 0.1443]],

         [[-0.1535]],

         [[ 0.1442]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.weight
tensor([[[[ 0.1686]],

         [[-0.1662]],

         [[-0.1287]]],


        [[[-0.1784]],

         [[-0.1708]],

         [[ 0.1272]]],


        [[[-0.1464]],

         [[ 0.1670]],

         [[ 0.1486]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.bias
tensor([ 0.11

In [7]:
    print ('*******************  reload saved ConvLSTM model  *********************')
    # load model parameters
    model_reload = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    model_reload.load_state_dict(torch.load(os.path.join(output_path, 'maximum_likelihood_convlstm_lorenz84.pkl'),
                                 map_location=device))
    # load entire model
    #model_reload = torch.load(os.path.join(output_path, 'sample_convlstm_lorenz84_0.pkl'))
    print(model_reload)

*******************  reload saved ConvLSTM model  *********************


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\maximum_likelihood_convlstm_lorenz84.pkl'

In [12]:
    print ('*******************  check the weight from reloaded ConvLSTM  *********************')
    for name, param in model_reload.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from reloaded ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[-0.1088]],

         [[-0.5378]],

         [[-0.0080]]],


        [[[-0.0202]],

         [[-0.3257]],

         [[-0.1092]]],


        [[[ 0.2254]],

         [[ 0.2367]],

         [[ 0.6049]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxi.bias
tensor([ 0.1563,  0.2719, -0.0114], device='cuda:0')
torch.Size([3])
cell0.Whi.weight
tensor([[[[-0.2003]],

         [[-0.0350]],

         [[-0.5474]]],


        [[[-0.0257]],

         [[ 0.0535]],

         [[-0.0233]]],


        [[[ 0.2967]],

         [[-0.0623]],

         [[ 0.1934]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.weight
tensor([[[[ 0.2479]],

         [[-0.0983]],

         [[-0.3932]]],


        [[[-0.1576]],

         [[-0.4027]],

         [[ 0.2763]]],


        [[[ 0.0457]],

         [[ 0.5174]],

         [[ 0.0108]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.bias
tens